# LwDM model

In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
import numpy as np
#import scipy
from time import time
from astropy import cosmology as cosmo
from astropy import units as u
from astropy import constants as c
from astropy.visualization import quantity_support
quantity_support()

In [ ]:
class Model(object):

    def __init__(self, Omega_tilde, w_tilde, Omega_L, z_min=1e-3, z_max=10):
        self.Omega_tilde = Omega_tilde
        self.w_tilde = w_tilde
        self.Omega_L = Omega_L
        self.z_min = z_min
        self.z_max = z_max
        
        step = np.log10(1+z_min)
        log_1zmax = np.log10(1+z_max)
        a = np.logspace(0, -log_1zmax, int(log_1zmax/step))
        self.z = 1/a[1:] - 1
        a_mid = np.sqrt(a[1:]*a[:-1])
        da = a[:-1] - a[1:]
        Omega_k = 1-Omega_tilde-Omega_L
        a_dot = np.sqrt(Omega_tilde*a_mid**(-1-3*w_tilde) + Omega_L*a_mid**2 + Omega_k)
        self.Dc_DH = np.cumsum(da/a_mid/a_dot)
        if np.abs(Omega_k) < 1e-6:
            #print(f'{Omega_k} is flat')
            self.DM_DH = self.Dc_DH
        elif Omega_k < 0:
            #print(f'{Omega_k} is closed')
            r = np.abs(np.sin(self.Dc_DH*np.sqrt(-Omega_k)))
            self.DM_DH = r/np.sqrt(-Omega_k)
        else:
            #print(f'{Omega_k} is open')
            r = np.sinh(self.Dc_DH*np.sqrt(Omega_k))
            self.DM_DH = r/np.sqrt(Omega_k)
        self.DL_DH = self.DM_DH*(1+self.z)

            
    def fit_mu(self, z, mu, err_mu):
        mu_model = 5*np.log10(np.interp(z, self.z, self.DL_DH)/1e-5) # units of Mpc
        
        posterior_error = 1/np.sum((1/err_mu)**2)
        best_norm = np.sum((mu-mu_model)/err_mu**2) * posterior_error
        self.chi2_mu = np.mean(((mu-mu_model-best_norm)/err_mu)**2)
        self.c_over_H0 = 10**(best_norm/5)
        self.H0 = 3e5/self.c_over_H0
        #print(f'c/H0 = {best_norm:.3g} Mpc, H0 = {3e5/best_norm:.4g} km/s/Mpc [{3e5/10**((best_norm+z.size*posterior_error)/5):.6g}, {3e5/10**((best_norm-z.size*posterior_error)/5):.6g}]')


In [ ]:
#model = Model(Omega_tilde, w_tilde, Omega_L)
model = Model(2, -1/3, 0)

In [ ]:
np.interp([0, 1, 2, 3], model.z, model.DL_DH) * 3e5/70

In [ ]:
#model = Model(Omega_tilde, w_tilde, Omega_L)
model = Model(.3, 0, .7)

In [ ]:
np.interp([0, 1, 2, 3], model.z, model.DL_DH) *3e5/70

Compute model grid:

In [ ]:
# LARGE GRID (Danger!)
Omega_tilde_models = np.linspace(0.0, 3.0, 101)
w_tilde_models = np.linspace(-0.67, 0.33, 51)
Omega_Lambda_models = np.linspace(-0.33, 1.17, 51)

def summary(x):
    return f'{x.size} values from {x[0]:.2f} to {x[-1]:.2f} (step={(x[-1]-x[0])/x.size:.2f})'

print('Omega_tilde:', summary(Omega_tilde_models))
print('w_tilde:', summary(w_tilde_models))
print('Omega_Lambda:', summary(Omega_Lambda_models))

In [ ]:
# SMALL GRID (memory-friendly ;^D)
Omega_tilde_models = np.linspace(0.0, 3.0, 51)
w_tilde_models = np.linspace(-0.67, 0.33, 26)
Omega_Lambda_models = np.linspace(-0.33, 1.17, 26)

def summary(x):
    return f'{x.size} values from {x[0]:.2f} to {x[-1]:.2f} (step={(x[-1]-x[0])/x.size:.2f})'

print('Omega_tilde:', summary(Omega_tilde_models))
print('w_tilde:', summary(w_tilde_models))
print('Omega_Lambda:', summary(Omega_Lambda_models))

In [ ]:
t0 = time()
Omega_tilde_grid, w_tilde_grid, Omega_Lambda_grid = np.meshgrid(Omega_tilde_models, w_tilde_models, Omega_Lambda_models, indexing='ij')
Omega_k_grid = 1-Omega_tilde_grid-Omega_Lambda_grid

z_min=1e-3
z_max=10
step = np.log10(1+z_min)
log_1zmax = np.log10(1+z_max)
a_models = np.logspace(0, -log_1zmax, int(log_1zmax/step))
z_models = 1/a_models[1:] - 1
a_mid = np.sqrt(a_models[1:]*a_models[:-1])
a_dot = np.sqrt(
    Omega_tilde_grid[:,:,:,np.newaxis] * a_mid[np.newaxis,np.newaxis,np.newaxis,:]**(-1-3*w_tilde_grid[:,:,:,np.newaxis]) +
    Omega_Lambda_grid[:,:,:,np.newaxis] * a_mid[np.newaxis,np.newaxis,np.newaxis,:]**2 +
    Omega_k_grid[:,:,:,np.newaxis])
DC_DH_grid = np.cumsum((a_models[:-1]-a_models[1:])/a_mid/a_dot, axis=3)

print(f'Model grid {DC_DH_grid.shape}, {DC_DH_grid.nbytes/pow(2, 20)} Mb, computed in {time()-t0:.3g} s')
print(' ... Paranoy@ Rulz ;^D\n')

Interpolation:

In [ ]:
#DC_DH_grid = np.log(DC_DH_grid)  # log interpolation

def index_weight(sorted_array, value):
    index = np.searchsorted(sorted_array, value).clip(1, sorted_array.size-1)
    weight = ((value-sorted_array[index-1])/(sorted_array[index]-sorted_array[index-1])).clip(0, 1)
    print(f'{sorted_array[index-1]:.3g} < {value:.3g} < {sorted_array[index]:.3g}, index={index}, weight={weight:.3g}')
    return index, weight

def DM_DH_interp(Omega_tilde, w_tilde, Omega_Lambda):
    i_Ot, w_Ot = index_weight(Omega_tilde_models, Omega_tilde)
    i_wt, w_wt = index_weight(w_tilde_models, w_tilde)
    i_OL, w_OL = index_weight(Omega_Lambda_models, Omega_Lambda)
    value = w_Ot*w_wt*w_OL * DC_DH_grid[i_Ot, i_wt, i_OL]
    value += w_Ot*w_wt*(1-w_OL) * DC_DH_grid[i_Ot, i_wt, i_OL-1]
    value += w_Ot*(1-w_wt)*w_OL * DC_DH_grid[i_Ot, i_wt-1, i_OL]
    value += w_Ot*(1-w_wt)*(1-w_OL) * DC_DH_grid[i_Ot, i_wt-1, i_OL-1]
    value += (1-w_Ot)*w_wt*w_OL * DC_DH_grid[i_Ot-1, i_wt, i_OL]
    value += (1-w_Ot)*w_wt*(1-w_OL) * DC_DH_grid[i_Ot-1, i_wt, i_OL-1]
    value += (1-w_Ot)*(1-w_wt)*w_OL * DC_DH_grid[i_Ot-1, i_wt-1, i_OL]
    value += (1-w_Ot)*(1-w_wt)*(1-w_OL) * DC_DH_grid[i_Ot-1, i_wt-1, i_OL-1]
    #value = np.exp(value)  # log interpolation
    Omega_k = 1-Omega_tilde-Omega_Lambda
    if(Omega_k < -1e-13):
        value = np.sin(value)
    elif(Omega_k > 1e-13):
        value = np.sinh(value)
    return value

Sanity checks:

In [ ]:
print('Onion model:')
y1 = DM_DH_interp(2, -1/3, 0)
y2 = np.sin(np.log(1+z_models))
print(y1)
print(y2)
print(y1/y2)

In [ ]:
print('Flat LCDM:')
H0 = 70
LCDM = cosmo.FlatLambdaCDM(H0, .3)
print('Omega =', LCDM.Ode0, LCDM.Om0, LCDM.Onu0, LCDM.Ogamma0, LCDM.Ok0)

y1 = DM_DH_interp(LCDM.Om0, 0., LCDM.Ode0)
y2 = LCDM.angular_diameter_distance(z_models).to_value(c.c/LCDM.H0)*(1+z_models)
print(y1)
print(y2)
print(y1/y2)

In [ ]:
print('Planck18-like does *NOT* work because Omega_k=0 requires accounting for Omega_r = Omega_nu+Omega_gamma ~ 0.0015')
LCDM = cosmo.Planck18
print(f'Omegas: ({LCDM.Ode0:.3g}, {LCDM.Om0:.3g}, {LCDM.Onu0:.3g}, {LCDM.Ogamma0:.3g}, {LCDM.Ok0:.3g})')

y1 = DM_DH_interp(LCDM.Om0, 0., LCDM.Ode0)
y2 = LCDM.angular_diameter_distance(z_models).to_value(c.c/LCDM.H0)*(1+z_models)
print(y1)
print(y2)
print('Flat:', y1/y2)
print('Open:', y1/np.sinh(y2))

In [ ]:
def plot_model(Omega_tilde, w_tilde, Omega_L, a_models, ax, style):
    model = Model(Omega_tilde, w_tilde, Omega_L)

    ax[0, 0].plot(model.z, model.Dc_DH/model.z, style,
                  label=r'$(\tilde\Omega, \tilde w, \Omega_\Lambda)$'+f'= ({Omega_tilde:.2f}, {w_tilde:.2f}, {Omega_L:.2f})')
    ax[1, 0].plot(model.z, model.DL_DH/model.z, style,
                  label=r'$(\tilde\Omega, \tilde w, \Omega_\Lambda)$'+f'= ({Omega_tilde:.2f}, {w_tilde:.2f}, {Omega_L:.2f})')
    ax[2, 0].plot(model.z, model.DM_DH/model.z, style,
                  label=r'$(\tilde\Omega, \tilde w, \Omega_\Lambda)$'+f'= ({Omega_tilde:.2f}, {w_tilde:.2f}, {Omega_L:.2f})')


fig_name = 'distances'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(7, 10))
axes = fig.subplots(nrows=3, ncols=1, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )

plot_model(.3, 0, .7, a_models, axes, 'b--')
plot_model(2, -1/3, 0, a_models, axes, 'k-')
plot_model(1+1/np.pi, -1/3, 0, a_models, axes, 'k--')
#plot_model(1.2, -1/3, 0, a_models, axes, 'r--')
plot_model(0, 0, 0, a_models, axes, 'r--')
#Omega_L = .7; plot_model(2-Omega_L, -1/3, Omega_L, a_models, axes, 'k:')
#Omega_L = -.7; plot_model(2-Omega_L, -1/3, Omega_L, a_models, axes, 'k-.')

ax = axes[0, 0]
ax.set_ylabel('$D_C(z)\ H_0/cz$')
ax.set_ylim(.15, 1.5)
ax.set_yscale('log')
ax.yaxis.set_major_formatter('{x:g}')
ax.yaxis.set_minor_formatter('{x:g}')
ax.grid(alpha=.2)

ax = axes[1, 0]
ax.grid(alpha=.2)
ax.set_ylabel('$D_L(z)\ H_0/cz$')
ax.set_ylim(.35, 3.5)
ax.set_yscale('log')
ax.yaxis.set_major_formatter('{x:g}')
ax.yaxis.set_minor_formatter('{x:g}')
ax.grid(alpha=.2)
ax.legend()

ax = axes[2, 0]
ax.set_ylabel('$D_M(z)\ H_0/cz$')
ax.set_ylim(.15, 1.5)
ax.set_yscale('log')
ax.yaxis.set_major_formatter('{x:g}')
ax.yaxis.set_minor_formatter('{x:g}')
ax.grid(alpha=.2)

ax = axes[-1, 0]
ax.set_xlabel('z')
ax.set_xscale('log')

fig.suptitle(fig_name)
fig.tight_layout()
fig.savefig(f'{fig_name}.pdf')

# Data

In [ ]:
SN_z, SN_err_z, SN_mu, SN_err_mu = np.loadtxt('data/Pantheon+_Data.dat', usecols=(2, 3, 10, 11), unpack=True, skiprows=1)
GRB1 = np.loadtxt('data/GRB_RNN.dat', skiprows=1)
GRB2 = np.loadtxt('data/GRB_RNN_2.dat', skiprows=1)
GRB_z = np.concatenate([GRB1[:, 0], GRB1[:, 3], GRB1[:, 6], GRB2[:, 0]])
GRB_mu = np.concatenate([GRB1[:, 1], GRB1[:, 4], GRB1[:, 7], GRB2[:, 1]])
GRB_err_mu = np.concatenate([GRB1[:, 2], GRB1[:, 5], GRB1[:, 8], GRB2[:, 2]])
QSO_z, QSO_mu, QSO_err_mu = np.loadtxt('data/table3QSO.dat', usecols=(3, 11, 12), unpack=True, skiprows=1)
#QSO_err_mu *= 2

def Dl_Mpc(mu):
    return 1e-5 * 10**(.2*mu)

SN_Dl = Dl_Mpc(SN_mu)
GRB_Dl = Dl_Mpc(GRB_mu)
QSO_Dl = Dl_Mpc(QSO_mu)


all_z = np.concatenate([SN_z, QSO_z, GRB_z])
all_mu = np.concatenate([SN_mu, QSO_mu, GRB_mu])
all_err_mu = np.concatenate([SN_err_mu, QSO_err_mu, GRB_err_mu])

#all_z = SN_z
#all_mu = SN_mu
#all_err_mu = SN_err_mu
#all_z = QSO_z
#all_mu = QSO_mu
#all_err_mu = QSO_err_mu
#all_z = GRB_z
#all_mu = GRB_mu
#all_err_mu = GRB_err_mu


In [ ]:
thetaBAO = np.array([
    0.11,19.8,3.26,
    0.235,9.06,0.23,
    0.365,6.33,0.22,
    0.45,4.77,0.17,
    0.47,5.02,0.25,
    0.49,4.99,0.21,
    0.51,4.81,0.17,
    0.53,4.29,0.30,
    0.55,4.25,0.25,
    0.57,4.59,0.36,
    0.59,4.39,0.33,
    0.61,3.85,0.31,
    0.63,3.90,0.43,
    0.65,3.55,0.16,
    2.225,1.77,0.31])

theta_z = thetaBAO[::3]
theta_err_z = np.array([0.005,0.035,0.025,0.01,0.005,0.01,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.01,0.025])
theta_angle_deg = thetaBAO[1::3]
theta_relative_err = thetaBAO[2::3] / theta_angle_deg
theta_DM = 180/np.pi/theta_angle_deg

# Comparison

In [ ]:
DM_DH_onion = np.sin(np.log(1+z_models))
DL_DH_onion = DM_DH_onion*(1+z_models)

def old_plot_DL_DH(z, dl, err_mu, **kwargs):
    y = dl/z
    error_y = y*err_mu/2
    model = np.interp(z, z_models, DL_DH_onion)/z # it was divided by z
    model_data = np.sum(model*y/error_y**2)
    model_model = np.sum(model*model/error_y**2)
    norm = model_data/model_model
    chi2 = np.mean(((y-model*norm)/error_y)**2)
    print(f'c/H0 = {norm:.3g} Mpc, H0 = {3e5/norm:.3g} km/s/Mpc, chi2={chi2:.3g}')
    ax.errorbar(z, y/norm, yerr=error_y/norm, **kwargs)

def plot_DL_DH(model, z, mu, err_mu, ax, style, **kwargs):
    mu_model = 5*np.log10(np.interp(z, model.z, model.DL_DH)/1e-5) # units of Mpc
    posterior_error = 1/np.sum((1/err_mu)**2)
    best_norm = np.sum((mu-mu_model)/err_mu**2) * posterior_error
    chi2 = np.mean(((mu-mu_model-best_norm)/err_mu)**2)
    print(f'best_norm = {best_norm:.3g} +- {posterior_error:.4g} km/s/Mpc, chi2={chi2:.4g}')
    norm = 10**(best_norm/5)
    print(f'c/H0 = {norm:.3g} Mpc, H0 = {3e5/norm:.4g} km/s/Mpc [{3e5/10**((best_norm+posterior_error)/5):.4g}, {3e5/10**((best_norm-posterior_error)/5):.4g}]')
    y = Dl_Mpc(mu-best_norm)/z
    dy_low = y-Dl_Mpc(mu-err_mu-best_norm)/z
    dy_high = Dl_Mpc(mu+err_mu-best_norm)/z-y
    ax.errorbar(z, y, yerr=(dy_low, dy_high), **kwargs)

    ax.plot(model.z, model.DL_DH/model.z, style,
            label=(f'$(\\tilde\\Omega, \\tilde w, \\Omega_\\Lambda)$=({model.Omega_tilde:.2f}, {model.w_tilde:.2f}, {model.Omega_L:.2f}):'
                   + f' h={3e5/norm:.1f}, $\\langle\\chi^2\\rangle$={chi2:.2f}'))

fig_name = 'comparison'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(10, 5))
axes = fig.subplots(nrows=1, ncols=1, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )

ax = axes[0, 0]
#ax.set_ylabel(r'$\mu$')
ax.set_ylabel(r'$D_L(z)\ H_0/cz$')
#ax.set_yscale('log')
ax.set_ylim(.35, 3.5)

#plot_DL_DH(SN_z, SN_mu, SN_err_mu, fmt='c.', alpha=.05)
#plot_DL_DH(QSO_z, QSO_mu, QSO_err_mu, fmt='y.', alpha=.1)
#plot_DL_DH(GRB_z, GRB_mu, GRB_err_mu, fmt='g.', alpha=.1)
#plot_DL_DH(all_z, all_mu, all_err_mu, fmt='k+', alpha=.1)
#plot_DL_DH(QSO_z, QSO_Dl, QSO_err_mu, fmt='y.', alpha=.2)
#plot_DL_DH(GRB_z, GRB_Dl, GRB_err_mu, fmt='g.', alpha=.1)
#plot_DL_DH(SN_z, SN_Dl, SN_err_mu, fmt='c.', alpha=.05)
#ax.plot(z_models, DL_DH_onion/z_models, 'k-')

onion = Model(2, -1/3, 0)
lcdm = Model(.3, 0, .7)
plot_DL_DH(Model(0, 0, 0), all_z, all_mu, all_err_mu, ax, 'b:', fmt='c+', alpha=.001)
plot_DL_DH(lcdm, all_z, all_mu, all_err_mu, ax, 'b-', fmt='c+', alpha=.001)
#plot_DL_DH(Model(.5, 0, .5), all_z, all_mu, all_err_mu, ax, 'b--', fmt='c+', alpha=.001)
#plot_DL_DH(Model(1, 0, 0), all_z, all_mu, all_err_mu, ax, 'b:', fmt='c+', alpha=.001)
plot_DL_DH(Model(1.+1/np.pi, -1/3, 0), all_z, all_mu, all_err_mu, ax, 'k--', fmt='c+', alpha=.001)
plot_DL_DH(onion, all_z, all_mu, all_err_mu, ax, 'k-', fmt='c+', alpha=.1)
#plot_DL_DH(Model(1.5, -1/3, .5), all_z, all_mu, all_err_mu, ax, 'r--', fmt='c+', alpha=.001)
'''
plot_DL_DH(lcdm, SN_z, SN_mu, SN_err_mu, ax, 'b-', fmt='c+', alpha=.1)
plot_DL_DH(lcdm, all_z, all_mu, all_err_mu, ax, 'b-', fmt='c+', alpha=.1)
plot_DL_DH(onion, SN_z, SN_mu, SN_err_mu, ax, 'k-', fmt='c+', alpha=.1)
plot_DL_DH(onion, all_z, all_mu, all_err_mu, ax, 'k-', fmt='c+', alpha=.1)
'''

ax.legend()

'''
ax = axes[1, 0]
ax.set_ylabel(r'$D_M / z r_d$')
ax.set_yscale('log')
#ax.set_ylim(.5, 50)

y = theta_DM/theta_z
norm = np.mean(np.interp(theta_z, z_models, DM_DH_onion) / y)
ax.errorbar(theta_z, y*norm, xerr=theta_err_z, yerr=theta_relative_err*y*norm, fmt='c.', alpha=.5)
ax.plot(z_models, DM_DH_onion/z_models, 'k-')
'''


ax = axes[-1, 0]
ax.set_xlabel(r'redshift')
ax.set_xscale('log')
ax.set_xlim(.001, 10)

for ax in axes.flatten():
    ax.tick_params(which='both', direction='in')
    ax.grid(alpha=.5)
fig.set_tight_layout(True)
plt.show()

In [ ]:
Omega_tilde_models = np.arange(0, 4, .01)
#Omega_tilde_models = np.logspace(-2, 2, 101)

fig_name = 'coasting-symmetric-flat'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(5, 5))
axes = fig.subplots(nrows=2, ncols=1, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )

def fit_family(model_params, obs, style, alpha):
    H0 = np.empty(model_params.shape[0])
    chi2 = np.empty(model_params.shape[0])
    for i, params in enumerate(model_params):
        model = Model(*params)
        model.fit_mu(*obs)
        H0[i] = model.H0
        chi2[i] = model.chi2_mu
    #return H0, chi2
    axes[0, 0].plot(Omega_tilde_models, H0, style, alpha=alpha)
    #axes[1, 0].plot(Omega_tilde_models, chi2, style, alpha=alpha)
    axes[1, 0].plot(Omega_tilde_models, chi2-np.nanmin(chi2), style, alpha=alpha)
    #axes[2, 0].plot(Omega_tilde_models, np.exp(-.5*(chi2-np.nanmin(chi2))*obs[0].size), style, alpha=alpha)


family = np.vstack([Omega_tilde_models, np.zeros_like(Omega_tilde_models)-1/3, np.zeros_like(Omega_tilde_models)]).T
fit_family(family, (SN_z, SN_mu, SN_err_mu), 'k-', 0.3)
fit_family(family, (QSO_z, QSO_mu, QSO_err_mu), 'k--', 0.3)
fit_family(family, (GRB_z, GRB_mu, GRB_err_mu), 'k:', 0.5)
#fit_family(family, (all_z, all_mu, all_err_mu), 'k-', 1)

family = np.vstack([Omega_tilde_models, np.zeros_like(Omega_tilde_models)-1/3, 2-Omega_tilde_models]).T
#fit_family(family, (SN_z, SN_mu, SN_err_mu), 'r-', 0.3)
#fit_family(family, (QSO_z, QSO_mu, QSO_err_mu), 'r--', 0.3)
#fit_family(family, (GRB_z, GRB_mu, GRB_err_mu), 'r:', 0.5)
#fit_family(family, (all_z, all_mu, all_err_mu), 'r-', 1)

family = np.vstack([Omega_tilde_models, np.zeros_like(Omega_tilde_models), 1-Omega_tilde_models]).T
#fit_family(family, (SN_z, SN_mu, SN_err_mu), 'b-', 0.3)
#fit_family(family, (QSO_z, QSO_mu, QSO_err_mu), 'b--', 0.3)
#fit_family(family, (GRB_z, GRB_mu, GRB_err_mu), 'b:', 0.5)
#fit_family(family, (all_z, all_mu, all_err_mu), 'b-', 1)

for ax in axes.flatten():
    ax.tick_params(which='both', direction='in')
    ax.grid(alpha=.5)
    #ax.axvline(.3, c='b', ls=':', label='Om = 0.3')
    ax.axvline(2, c='k', ls='--', label='This work')

ax = axes[0, 0]
ax.set_ylabel(r'$H_0$')
ax.legend()
ax = axes[1, 0]
ax.set_ylabel(r'$\langle\chi^2\rangle$')
ax.set_ylim(-.1, 1.1)
#ax.set_yscale('log')
#ax = axes[2, 0]
#ax.set_ylabel(r'$dp/d\tilde\Omega$')
ax.set_xlabel(r'$\tilde\Omega$')
#ax.legend()
#ax.set_xscale('log')

fig.set_tight_layout(True)
plt.show()

In [ ]:
SN_z.size, QSO_z.size, GRB_z.size, all_z.size

In [ ]:
5*np.log10(np.interp(SN_z, z_models, DL_DH_onion)/1e-5)

In [ ]:
SN_mu

In [ ]:
5*np.log10(4200)